# 1.Zaimplementuj metodę AMS dla 3 z zmiennych.

In [30]:
import random

In [31]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i']

In [32]:
n=10000

In [33]:
stream = random.choices(letters, k = n)

In [34]:
elements=list(set(stream))

In [35]:
S_2=sum([stream.count(i)**2 for i in elements])
print('Drugi moment rzeczywisty :',S_2)

Drugi moment rzeczywisty : 11125568


In [36]:
t = random.choices([i for i in range(n)], k = 3 )
print(t)

[7381, 1463, 6997]


In [37]:
counter = [(stream[i], stream[i:].count(stream[i])) for i in t]
print(counter)

[('b', 273), ('g', 974), ('h', 332)]


In [38]:
S_est = sum([n*(2*i[1]-1) for i in counter])/len(counter)
print('Drugi moment estymowany :',S_est)

Drugi moment estymowany : 10516666.666666666


# 2. Przetestuj na przykładzie prezentowanym na wykładzie.

In [39]:
przyklad = ['a','b','c','b','d','a','c','d','a','b','d','c','a','a','b']

In [40]:
n = len(przyklad)

In [41]:
elementy=list(set(przyklad))

In [42]:
#  drugi moment
S_2=sum([przyklad.count(i)**2 for i in elementy])

In [43]:
# czas (3,8,13) --> (2,7,12), bo numeracja od 0
t = [2, 7, 12]  

In [44]:
counter=[(przyklad[i],przyklad[i:].count(przyklad[i])) for i in t] # zmienna i liczebnosc

In [45]:
counter

[('c', 3), ('d', 2), ('a', 2)]

In [46]:
S_est= sum([n*(2*i[1]-1) for i in counter])/len(counter) 
print('Drugi moment rzeczywisty :',S_2)
print('Drugi moment estymowany :',S_est)

Drugi moment rzeczywisty : 59
Drugi moment estymowany : 55.0


# 3. (*) Zaimplementuj metodę AMS dla K dowolnych zmiennych (K będzie małe).


In [201]:
class AMS:
  def __init__(self, k=4):
    self.SAMPLE_COUNT = k+1
    self.sample = []
    self.sample_index = []
    self.index = 0 
    self.S_2 = 0
    self.S_2_dict = {} #słownik na elementy i ich counter 
    self.S_est = 0
    self.S_est_dict = {}
    self.dict_sample_index = {}

  def RS_update(self, new_sample):
    self.index += 1

    if self.index < self.SAMPLE_COUNT:
      self.sample.append(new_sample)
      self.sample_index.append(self.index)
      self.dict_sample_index[new_sample]  = self.index

    else:
      # Losowo zamieniaj element w rezerwuarze
      # z malejącym prawdopodobieństwem.
      # wylosuj liczbę z zakresu od 0 do index
      r = random.randint(0, self.index)

      # Jeśli pozycja w zakresie rezerwuaru to zamień wylosowany element
      if r < self.SAMPLE_COUNT-1:
        self.sample[r] = new_sample
        self.sample_index[r] = self.index #!!! ZMIANA BO CHCEMY DOSTAĆ INDEKSY
        self.dict_sample_index[new_sample]  = self.index

  def real(self,new_sample):
    #jeśli nie było jeszcze takiej obserwacji, to dodaj nowy klucz
    if new_sample not in self.S_2_dict.keys():
      self.S_2_dict[new_sample] = 1
    #jeśli jest to zlicz kolejną wartość
    else:
      self.S_2_dict[new_sample] += 1

    self.S_2 = sum([self.S_2_dict[i]**2 for i in self.S_2_dict])

  def estimation(self, new_sample):
    #jeśli nowa obserwacja jest na obu listach
    if (new_sample in self.dict_sample_index.keys()) and (new_sample in self.sample):
      #jeśli nie było jeszcze takiej obserwacji, to dodaj nowy klucz do słownika na estymację
      if new_sample not in self.S_2_dict.keys():
        self.S_est_dict[new_sample] = 1
      #jeśli jest to zlicz kolejną wartość
      else:
        self.S_est_dict[new_sample] += 1
      #jeśli obserwacji tej nie ma na obu listach, to nas nie interesuje do zliczania
    else:
      #jeśli wcześniej nas interesowała to trzeba ją usunąć 
      if new_sample in self.S_est_dict.keys():
          self.S_est_dict.pop(new_sample, None)
    self.S_est = sum([self.index*(2*self.S_est_dict[i]-1) for i in self.S_est_dict])

In [203]:
ams = AMS()
for element in przyklad:
  ams.RS_update(element)
  ams.estimation(element)
  ams.real(element)
print(ams.sample, ams.S_est_dict, ams.S_2_dict, ams.dict_sample_index)
print('Drugi moment rzeczywisty :',ams.S_2)
print('Drugi moment estymowany :',ams.S_est)

['a', 'b', 'd', 'd'] {'a': 5, 'b': 4, 'd': 3} {'a': 5, 'b': 4, 'c': 3, 'd': 3} {'a': 9, 'b': 10, 'c': 7, 'd': 11}
Drugi moment rzeczywisty : 59
Drugi moment estymowany : 315
